In [1]:
import os
from openaikey import key

os.environ["OPENAI_API_KEY"] = str(key())

In [2]:
# !pip install dspy-ai

In [3]:
import pandas as pd

df = pd.read_csv("hf://datasets/fka/awesome-chatgpt-prompts/prompts.csv")

C:\Users\ayush\Downloads\Superteams work\colbertvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df.head()

,act,prompt
0,Linux Terminal,I want you to act as a linux terminal. I will ...
1,English Translator and Improver,"I want you to act as an English translator, sp..."
2,`position` Interviewer,I want you to act as an interviewer. I will be...
3,JavaScript Console,I want you to act as a javascript console. I w...
4,Excel Sheet,I want you to act as a text based excel. you'l...


In [5]:
df.shape

(153, 2)

In [6]:
import dspy

dataset = []

for act, prompt in df.values:
    dataset.append(dspy.Example(question=act,gold_reasoning=prompt,answer=prompt).with_inputs("question"))

print(dataset[:3])

[Example({'question': 'Linux Terminal', 'gold_reasoning': 'I want you to act as a linux terminal. I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. do not write explanations. do not type commands unless I instruct you to do so. when i need to tell you something in english, i will do so by putting text inside curly brackets {like this}. my first command is pwd', 'answer': 'I want you to act as a linux terminal. I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. do not write explanations. do not type commands unless I instruct you to do so. when i need to tell you something in english, i will do so by putting text inside curly brackets {like this}. my first command is pwd'}) (input_keys={'question'}), Example({'question': 'English Translator and

In [7]:
import dspy
from dspy.datasets.gsm8k import  gsm8k_metric

# Set up the LM.
turbo = dspy.OpenAI(model='gpt-3.5-turbo', max_tokens=250)
dspy.settings.configure(lm=turbo)

# Load math questions from the GSM8K dataset.
# Split the dataset into training and dev sets
train_size = int(0.8 * len(dataset))  # 80% for training
dev_size = len(dataset) - train_size   # Remaining 20% for dev

dataset_trainset = dataset[:train_size]
dataset_devset = dataset[train_size:]

print("Train set:", dataset_trainset[:3])
print("Dev set:", dataset_devset[:3])

Train set: [Example({'question': 'Linux Terminal', 'gold_reasoning': 'I want you to act as a linux terminal. I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. do not write explanations. do not type commands unless I instruct you to do so. when i need to tell you something in english, i will do so by putting text inside curly brackets {like this}. my first command is pwd', 'answer': 'I want you to act as a linux terminal. I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. do not write explanations. do not type commands unless I instruct you to do so. when i need to tell you something in english, i will do so by putting text inside curly brackets {like this}. my first command is pwd'}) (input_keys={'question'}), Example({'question': 'English Tra

In [8]:
# print(dataset_trainset)

In [9]:
cache_turn_on = os.environ.get('DSP_CACHEBOOL', 'True').lower() != 'false'

In [10]:
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought("question -> answer")
    
    def forward(self, question):
        return self.prog(question=question)

In [11]:
from dspy.teleprompt import BootstrapFewShot

# Set up the optimizer: we want to "bootstrap" (i.e., self-generate) 4-shot examples of our CoT program.
config = dict(max_bootstrapped_demos=8, max_labeled_demos=8)

# Optimize! Use the `gsm8k_metric` here. In general, the metric is going to tell the optimizer how well it's doing.
teleprompter = BootstrapFewShot(metric=gsm8k_metric, **config)
optimized_cot = teleprompter.compile(CoT(), trainset=dataset_trainset)

  8%|██████████▌                                                                                                                      | 10/122 [00:16<03:04,  1.65s/it]


Bootstrapped 8 full traces after 11 examples in round 0.


In [12]:
import os
from openaikey import key2

os.environ["OPENAI_API_KEY"] = str(key2())

In [13]:
from dspy.evaluate import Evaluate

# Set up the evaluator, which can be used multiple times.
evaluate = Evaluate(devset=dataset_devset, metric=gsm8k_metric, num_threads=10, display_progress=True, display_table=0)

# Evaluate our `optimized_cot` program.
evaluate(optimized_cot)

Average Metric: 21 / 31  (67.7): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 31/31 [00:05<00:00,  5.37it/s]


67.74

In [14]:
turbo.inspect_history(n=1)




Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: Linux Terminal
Reasoning: Let's think step by step in order to produce the answer. We need to break down the process into clear and concise steps to effectively guide the user through the task at hand.
Answer: I will provide you with step-by-step instructions on how to accomplish the specific task in the Linux terminal. Let's start by opening the terminal and executing the following commands to achieve the desired outcome.

---

Question: English Translator and Improver
Reasoning: Let's think step by step in order to produce the answer. We need to translate the given text into English and then improve the translation to make it more accurate and natural.
Answer: I want you to act as an English translator and improve the translation. I will provide you with a 

'\n\n\nGiven the fields `question`, produce the fields `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let\'s think step by step in order to ${produce the answer}. We ...\nAnswer: ${answer}\n\n---\n\nQuestion: Linux Terminal\nReasoning: Let\'s think step by step in order to produce the answer. We need to break down the process into clear and concise steps to effectively guide the user through the task at hand.\nAnswer: I will provide you with step-by-step instructions on how to accomplish the specific task in the Linux terminal. Let\'s start by opening the terminal and executing the following commands to achieve the desired outcome.\n\n---\n\nQuestion: English Translator and Improver\nReasoning: Let\'s think step by step in order to produce the answer. We need to translate the given text into English and then improve the translation to make it more accurate and natural.\nAnswer: I want you to act as an English translator and improve the translation

In [18]:
result = optimized_cot(question="What is the fullname of the person in the given document ?")

In [19]:
# Extract only the answer
answer = result.answer

# Print the extracted answer
print(answer)

I will carefully review the document you provided and extract the person's full name based on the information available. Let's start by analyzing the document to identify and confirm the individual's complete name.


In [20]:
optimized_cot.save("optimized_cot_bestprompter.json")

[('prog', Predict(StringSignature(question -> rationale, answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})
)))]
